In [ ]:
# Screener

In [1]:
import requests
import pandas as pd
import numpy as np
import yfinance as yf

from bs4 import BeautifulSoup
from sklearn.preprocessing import StandardScaler

In [2]:
class Stock:
    def __init__(self, ticker, sector):
        self.ticker = ticker
        self.sector = sector
        self.price = 0.0
        self.url = f"https://finance.yahoo.com/quote/{self.ticker}/key-statistics?p={self.ticker}"
        self.data = pd.DataFrame()
        # Deep Learning Attributes
        self.technical_indicators = pd.DataFrame()
        self.today_technical_indicators = pd.DataFrame()
        self.labels = pd.DataFrame()
        self.prediction = 0.0
        # Metrics
        self.metrics = {}
        self.metric_aliases = {
            'Market Cap (intraday)': 'market_cap',
            # other key-value pairs
        }


In [3]:
def get_historical(self):
        stock = yf.Ticker(self.ticker)
        history = stock.history(start='2010-01-01', end='2023-02-28')
        self.data = history

In [4]:

def add_technical_indicators(self):
        # get historical stock prices
        prices = self.data
        if len(prices) < 20:
            return
        
        # calculate 20-day moving average
        prices['MA20'] = prices['Close'].rolling(window=20).mean()
        
        # calculate 50-day moving average
        prices['MA50'] = prices['Close'].rolling(window=50).mean()
        
        # calculate relative strength index (RSI)
        delta = prices['Close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=14).mean()
        avg_loss = loss.rolling(window=14).mean()
        rs = avg_gain / avg_loss
        prices['RSI'] = 100 - (100 / (1 + rs))
        
        # calculate moving average convergence divergence (MACD)
        exp1 = prices['Close'].ewm(span=12, adjust=False).mean()
        exp2 = prices['Close'].ewm(span=26, adjust=False).mean()
        macd = exp1 - exp2
        signal = macd.ewm(span=9, adjust=False).mean()
        prices['MACD'] = macd - signal
        
        # calculate Bollinger Bands
        prices['20MA'] = prices['Close'].rolling(window=20).mean()
        prices['20STD'] = prices['Close'].rolling(window=20).std()
        prices['UpperBand'] = prices['20MA'] + (prices['20STD'] * 2)
        prices['LowerBand'] = prices['20MA'] - (prices['20STD'] * 2)

        # Features for deep learning model
        train_data_aux = prices[['Close', 'MA20', 'MA50', 'RSI', 'MACD', 'UpperBand', 'LowerBand']].dropna()
        self.technical_indicators = train_data_aux.iloc[:-10, :].drop('Close', axis=1)

        # Set label as profit loss of 10 day future price from actual price
        labels_aux = (train_data_aux['Close'].shift(-10) > train_data_aux['Close']).astype(int)
        self.label =  labels_aux[:-10]

        # Today features for prediction
        self.today_technical_indicators = prices[['MA20', 'MA50', 'RSI', 'MACD', 'UpperBand', 'LowerBand']].iloc[-1,:] 

        prices = prices.reset_index()
        # store technical indicators in stock data dictionary
        self.data.update(prices[['Date', 'MA20', 'MA50', 'RSI', 'MACD', 'UpperBand', 'LowerBand']].to_dict('list'))

In [6]:
# Train deep learning models on selected stocks
def train_models(self):
    # Get data for training and testing
    filtered_stocks = self.apply_filters()

    for stock in filtered_stocks:
        train_data = stock.technical_indicators
        train_labels = stock.label

        # Normalize the data
        train_data = self.scaler.fit_transform(train_data)
        train_labels = np.array(train_labels)

        # Create and train model
        model = create_model(train_data)
        model.fit(train_data, train_labels, epochs=10)
        self.models[stock.ticker] = model

In [7]:
# Predict whether new stocks will pass filters
def predict_stocks(self, new_stocks):
    # Add technical indicators to new stocks
    for stock in new_stocks:
        stock.get_historical()
        stock.add_technical_indicators()

    # Make predictions for each stock using its corresponding model
    predicted_stocks = []
    for stock in new_stocks:
        if stock.ticker in self.models:
            model = self.models[stock.ticker]
            # Reshape as there is only one sample
            new_features_aux = np.array(stock.today_technical_indicators).reshape(1, -1)
            new_stock_data = self.scaler.fit_transform(new_features_aux)
            prediction = model.predict(new_stock_data)
            stock.prediction = prediction
            if prediction > 0.5:
                predicted_stocks.append(stock)

    return predicted_stocks

In [8]:
import tensorflow as tf

# Simple Dense model 
def create_model(train_data):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(train_data.shape[1],), activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

2024-01-30 19:04:41.702203: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 19:04:41.749483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 19:04:41.749516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 19:04:41.750917: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 19:04:41.757773: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 19:04:41.758434: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
def filter_technical_indicator(stock, indicator_name, operator, value):
    if indicator_name not in stock.today_technical_indicators:
        return False

    # Obtain the value of the technical indicator
    indicator_value = stock.today_technical_indicators[indicator_name]
    
    # Check if the value is 'price':
    if value == 'price':
        value = float(stock.price)
    else:
        value = float(value)
    
    # Compare according to operator
    if operator == '>':
        return float(indicator_value) > value
    elif operator == '>=':
        return float(indicator_value) >= value
    elif operator == '<':
        return float(indicator_value) < value
    elif operator == '<=':
        return float(indicator_value) <= value
    elif operator == '==':
        return float(indicator_value) == value
    else:
        return False

In [12]:
# Run example with 2 stocks
filters = [lambda stock: filter_sector(stock, 'Interactive Media & Services'),
           lambda stock: filter_price(stock, 60, 200),
           lambda stock: filter_metric(stock, 'profit_margin', '>', 15),
           lambda stock: filter_technical_indicator(stock, 'UpperBand', '>', 'price'),
           lambda stock: filter_technical_indicator(stock, 'LowerBand', '<', 'price'),
]

sp500_stocks = [Stock('GOOGL', 'Interactive Media & Services'), 
                Stock('GOOG', 'Interactive Media & Services')
]

# Screener
screener = Stock(sp500_stocks, filters)
# Add Data
screener.add_data()
# Apply Filters
filtered_stocks = screener.apply_filters()
# Train model
screener.train_models()
# Make predictions
predicted_stocks = screener.predict_stocks(filtered_stocks)

AttributeError: 'Stock' object has no attribute 'add_data'